In [1]:
import gmsh
import sys
from CompositeSandwich.box_labeling import box_labeling

gmsh.initialize()

# Crear el modelo
gmsh.model.add("Anisotropic_Box")

# Dimensiones de la caja
Lx, Ly, Lz = 10, 10, 2

# Crear la caja
box = gmsh.model.occ.addBox(0, 0, 0, Lx, Ly, Lz)
gmsh.model.occ.synchronize()

ph = gmsh.model.addPhysicalGroup(3, [box])
gmsh.model.setPhysicalName(3, ph, "box")

box_labeling(box, "box")
# Crear la malla
gmsh.model.mesh.generate(3)

# quadratic
gmsh.model.mesh.setOrder(2)
# Guardar la malla en formato CalculiX
gmsh.write("anisotropic_box.inp")

# Finalizar Gmsh
gmsh.finalize()


In [2]:
from djccx.inp.inp import  inp


In [3]:
inp_f = inp("anisotropic_box.inp")

In [4]:
E1 = 100
E2 = 10
E3 = 10
nu12 = 0.3
nu13 = 0.3
nu23 = 0.3
G12 = 5
G13 = 5
G23 = 5

material = inp_f.CreateElasticEngineeringMaterial("MAT1", E1, E2, E3, nu12, nu13, nu23, G12, G13, G23)

In [5]:
inp_f.cards

array([Card (None) :*HEADING, Card (*NODE) :*NODE,
       Card (*ELEMENT) :SURFACE1, Card (*ELEMENT) :SURFACE2,
       Card (*ELEMENT) :SURFACE3, Card (*ELEMENT) :SURFACE4,
       Card (*ELEMENT) :SURFACE5, Card (*ELEMENT) :SURFACE6,
       Card (*ELEMENT) :VOLUME1, Card (*ELSET) :BOX_X0,
       Card (*ELSET) :BOX_XL, Card (*ELSET) :BOX_Y0,
       Card (*ELSET) :BOX_YL, Card (*ELSET) :BOX_Z0,
       Card (*ELSET) :BOX_ZL, Card (*ELSET) :BOX, Card (*MATERIAL) :MAT1],
      dtype=object)

In [6]:
orientation = inp_f.CreateOrientation("OR1",[1,0,0],[0,1,0])

elset_bot = inp_f.select("BOX_Y0","elset")
elset_top = inp_f.select("BOX_YL","elset")

x_min = inp_f.select("BOX_X0","elset")
x_max = inp_f.select("BOX_XL","elset")

nset_bot = inp_f.CreateNsetFromElset(elset_bot,"nset_bot")
nset_top = inp_f.CreateNsetFromElset(elset_top,"nset_top")

nset_x_min = inp_f.CreateNsetFromElset(x_min,"nset_x_min")
nset_x_max = inp_f.CreateNsetFromElset(x_max,"nset_x_max")


inp_f.remove_by_type(2)

allelset = inp_f.CreateElsetAll()

In [7]:
inp_f.CreateSolidSection(allelset,material,orientation=orientation)

step = inp_f.CreateStaticStep()

step.CreateBoundary(nset_bot,2,0.0)
step.CreateBoundary(nset_top,2,1.0)

# remove 2d 
step.CreateBoundary(nset_x_min,1,0.0)
step.CreateBoundary(nset_x_max,1,0.0)



*BOUNDARY
NSET_X_MAX, 1, 0.0

In [8]:
inp_f.cards.tolist()

[Card (None) :*HEADING,
 Card (*NODE) :*NODE,
 Card (*ELEMENT) :VOLUME1,
 Card (*ELSET) :BOX_X0,
 Card (*ELSET) :BOX_XL,
 Card (*ELSET) :BOX_Y0,
 Card (*ELSET) :BOX_YL,
 Card (*ELSET) :BOX_Z0,
 Card (*ELSET) :BOX_ZL,
 Card (*ELSET) :BOX,
 Card (*MATERIAL) :MAT1,
 Card (*ORIENTATION) :OR1,
 Card (*NSET) :NSET_BOT,
 Card (*NSET) :NSET_TOP,
 Card (*NSET) :NSET_X_MIN,
 Card (*NSET) :NSET_X_MAX,
 Card (*ELSETOFELSET) :ALL,
 Card (*SOLIDSECTION) :SOLID_SECTION,
 Card (*STEP) :STEP]

In [9]:
inp_f.print("ccx.inp")

In [10]:
inp_f.run("output")

Running Calculix at:  output
Output file:  c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\scripts\T06_ortotropo\output\out.txt
Command:
 mpiexec -n 4 c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\.conda\Lib\site-packages\djccx\bin\ccx_dynamic.exe main
Error reading cvf file

pid:  8780 

Calculix finished



{'data':       node          x          y    z            D1        D2        D3  \
 node                                                                      
 1        1   0.000000   0.000000  2.0  0.000000e+00  0.000000 -0.062846   
 2        2   0.000000   0.000000  0.0  0.000000e+00  0.000000 -0.000485   
 3        3   0.000000  10.000000  2.0  0.000000e+00  1.000000 -0.062846   
 4        4   0.000000  10.000000  0.0  0.000000e+00  1.000000 -0.000485   
 5        5  10.000000   0.000000  2.0  0.000000e+00  0.000000 -0.062846   
 ...    ...        ...        ...  ...           ...       ...       ...   
 1031  1031   0.662605   7.846230  0.5  3.318850e-19  0.784623 -0.016076   
 1032  1032   7.843540   9.340170  1.5  6.363460e-17  0.934017 -0.047256   
 1033  1033   7.852940   0.656582  1.5 -9.234870e-17  0.065658 -0.047256   
 1034  1034   4.985050   0.648127  1.5 -1.599020e-16  0.064813 -0.047256   
 1035  1035   3.305420   4.282630  1.0  2.432090e-17  0.428263 -0.031666   
 
  